Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [2]:
%pip install azureml-widgets
%pip install xgboost==1.5.0
%pip install --upgrade azureml-sdk[notebooks,automl]
%pip install azureml-train-automl-runtime==1.57.0
%pip install azureml-pipeline-core==1.57.0
%pip install azureml-pipeline-steps

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 16, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 KB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 KB 35.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-eed4ac5a-efce-4eda-af2e-a99a2ab8f2da
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: isodate
    Found existing installation: isodate 0.6.0
    Not uninstalling isodate at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-eed4ac5a-efce-4eda-af2e-a99a2ab8f2da
    Can't unin

In [3]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 22, Finished, Available, Finished)

SDK version: 1.57.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [4]:
ws = Workspace.from_config(path="Users/odl_user_268740/starter_files/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 23, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DM873AA7L to authenticate.
Interactive authentication successfully completed.
quick-starts-ws-268740
aml-quickstarts-268740
westeurope
610d6e37-4747-4a20-80eb-3aad70a55f43


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [5]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-project-2'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 24, Finished, Available, Finished)

Name,Workspace,Report Page,Docs Page
ml-experiment-project-2,quick-starts-ws-268740,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 25, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 26, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c', 'run_id': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [8]:
dataset.take(5).to_pandas_dataframe()

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 27, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c', 'run_id': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 28, Finished, Available, Finished)

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [10]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 29, Finished, Available, Finished)

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 30, Finished, Available, Finished)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 31, Finished, Available, Finished)

In [13]:
pipeline_run = experiment.submit(pipeline)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 32, Finished, Available, Finished)

Created step automl_module [eccccb35][4fb02ff5-c54c-4ad0-978c-2a3a607b2461], (This step will run and generate new outputs)
Submitted PipelineRun bffa589d-db93-4962-9a02-c97e5c3259b3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bffa589d-db93-4962-9a02-c97e5c3259b3?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268740/workspaces/quick-starts-ws-268740&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [14]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 33, Finished, Available, Finished)

Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


Initializing logging file for interpret-community


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [15]:
pipeline_run.wait_for_completion()

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 34, Finished, Available, Finished)

PipelineRunId: bffa589d-db93-4962-9a02-c97e5c3259b3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bffa589d-db93-4962-9a02-c97e5c3259b3?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268740/workspaces/quick-starts-ws-268740&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: bbf7eadd-f1c4-4100-953f-931da4dfd4f6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bbf7eadd-f1c4-4100-953f-931da4dfd4f6?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-268740/workspaces/quick-starts-ws-268740&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [16]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 35, Finished, Available, Finished)

Downloaded azureml/bbf7eadd-f1c4-4100-953f-931da4dfd4f6/metrics_data, 1 files out of an estimated total of 1


In [17]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 36, Finished, Available, Finished)

,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_8,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_9,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_16,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_7,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_10,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_18,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_14,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_12,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_20,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_25,...,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_0,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_2,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_1,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_4,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_5,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_3,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_30,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_11,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_32,bbf7eadd-f1c4-4100-953f-931da4dfd4f6_31
average_precision_score_micro,[0.9799703642815116],[0.8455861844577244],[0.8847988203557676],[0.9760584861374465],[0.9752498455464272],[0.9797798706773968],[0.97707236221129],[0.9796736019290664],[0.9671420339074859],[0.9800356198767971],...,[0.9805151927136845],[0.9586199524254295],[0.9793987356897456],[0.9744889524228721],[0.9789389178388146],[0.9633554731010853],[0.9793702498898297],[0.8100423802856577],[0.9746752430107849],[0.980346014408223]
f1_score_weighted,[0.9048597237768088],[0.8049757322856604],[0.8659213543958487],[0.9013350533065821],[0.8953324743236205],[0.9098016443897835],[0.907626910573524],[0.9087485655110661],[0.8907906149634255],[0.9092400519650629],...,[0.9072831557855962],[0.8742539622850937],[0.9087877883554606],[0.8909573471197098],[0.899959550454415],[0.8353395018439429],[0.8886031510001888],[0.8025702955621553],[0.9074064959938667],[0.9103870858100789]
AUC_macro,[0.941632999720291],[0.8736002978621721],[0.9229976271054576],[0.9290011799639528],[0.9237121814143637],[0.9415278773430249],[0.9330361194931156],[0.9405160165750666],[0.9207719965101223],[0.9418122171652339],...,[0.9446537630106308],[0.8675078309224651],[0.942328104073932],[0.922536848403344],[0.9388252597495217],[0.8782817168568131],[0.9424031253299546],[0.8376067663615803],[0.9256488412457604],[0.9450492454343544]
log_loss,[0.19932096535538657],[0.5888782985898102],[0.40290230966751145],[0.21382270165851136],[0.21235370304099976],[0.19693610296079414],[0.22624716798653413],[0.21143958424122453],[0.22630538933197097],[0.20047350822493099],...,[0.17851374134751752],[0.25851181222075476],[0.18692041784468305],[0.22090578349014284],[0.20462013175825502],[0.26899983990507864],[0.19986862844075845],[0.5503501121228047],[0.567904751136231],[0.1804632385841266]
balanced_accuracy,[0.7210524463412782],[0.7812959042098966],[0.8539734406229913],[0.6863829010812322],[0.6653862112783807],[0.7462730180958679],[0.7099187362345256],[0.7324051999918495],[0.6653500899328884],[0.7207468041871123],...,[0.7445642005975768],[0.5910795095647471],[0.7477868729473351],[0.6457565754741621],[0.693976256235563],[0.5],[0.6379682576730074],[0.7484699368524785],[0.7413535686963157],[0.7565884407989671]
weighted_accuracy,[0.9574977663473424],[0.7610772903031019],[0.8411526027126678],[0.9679994692811393],[0.9686663170697974],[0.9546457273395061],[0.9670810589128979],[0.9588464085130249],[0.9607196759303707],[0.9647715810627646],...,[0.9512815952194833],[0.9772648594095156],[0.952375685487909],[0.9720244705544736],[0.9619470207827714],[0.9843450583187134],[0.9724427450812216],[0.7673328974294973],[0.9528363853308665],[0.9513270098533659]
matthews_correlation,[0.5016093372462171],[0.38620790341654526],[0.5238288494614208],[0.47805766735773875],[0.444201567492641],[0.5328108142609521],[0.5134353928895572],[0.5231250045786537],[0.41778310629483134],[0.5229195744551492],...,[0.5216286298277554],[0.3332520888241266],[0.52924365161229],[0.42149687086703563],[0.47077965319103915],[0.0],[0.408491687757675],[0.34617322638862885],[0.5206602954529022],[0.5398012737528465]
recall_score_micro,[0.9104704097116844],[0.7650986342943854],[0.8437025796661608],[0.9119878603945372],[0.9083459787556905],[0.91320182

### Retrieve the Best Model

In [18]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 37, Finished, Available, Finished)

Downloaded azureml/bbf7eadd-f1c4-4100-953f-931da4dfd4f6/model_data, 1 files out of an estimated total of 1


In [19]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 38, Finished, Available, Finished)

[08:20:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[08:20:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[08:20:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/synfs/notebook/0/aml_notebook_mount')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('26', Pipeline(steps=[('maxabsscaler', MaxAbs...pu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))]))], flatten_transform=False, weights=[0.26666666666666666, 0.2, 0.06666666666666667, 0.2, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [20]:
best_model.steps

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 39, Finished, Available, Finished)

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('26', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='goss', colsample_bytree=0.8911111111111111, learning_rate=0.03158578947368421, max_bin=280, max_depth=9, min_child_weight=9, min_data_in_leaf=1e-05, min_split_gain=0.2631578947368421, n_estimators=100, n_jobs=1, num_leaves=89, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None, reg_alpha=0.10526315789473684, reg_lambda=0.21052631578947367, subsample=1))], verbose=False)), ('31', Pipeline(memory=None, steps=[('stand

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [21]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 40, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '1bfed5b4-9ce5-4283-b210-078a1225f06c', 'run_id': '1bfed5b4-9ce5-4283-b210-078a1225f06c'}


#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [22]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 41, Finished, Available, Finished)

In [23]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 42, Finished, Available, Finished)

,0,1
0,28911,347
1,1306,2386


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [24]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 43, Finished, Available, Finished)

Name,Id,Status,Endpoint
Bankmarketing Train,07d4ac05-9fa9-4e80-8203-02e7f4896167,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [25]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 44, Finished, Available, Finished)

Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [26]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 45, Finished, Available, Finished)

In [27]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 46, Finished, Available, Finished)

Submitted pipeline run:  bec858e4-1b28-428d-98d0-a9078a2f0b0a


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [28]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

StatementMeta(968a7889-7dac-456c-85d8-8abf2831c4eb, 0, 47, Finished, Available, Finished)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …